In [1]:
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text
import random

from utils.data_loaders import get_wake_datasets
from utils.evaluation import evaluate_model
import utils.plotting as plotting

In [2]:
FACTORS_FOLDER = "discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01"
DATA_FOLDER = f"data/{FACTORS_FOLDER}/"
INPUT_VAR_TO_TRAIN_REDUCTION_FACTOR = {'ti': 4, 'ct': 4, 'x/D': 4, 'y/D': 4}
train_reduc_factor_string = "training_factors=" + "-".join([f"{k}{v}" for k, v in INPUT_VAR_TO_TRAIN_REDUCTION_FACTOR.items()])
#INPUT_VAR_TO_TRAIN_RANGES = {'ti': [(0.15, 0.4)], 'ct': [(0.3, 0.7)]}
#train_range_string = "training_ranges=" + '-'.join([f"{var}{r[0]}-{r[1]}" for var, ranges in INPUT_VAR_TO_TRAIN_RANGES.items() for r in ranges])
CONSIDER_WS = False
COORDS_AS_INPUT = True # univariate setting
MAX_DEPTH = None

MODEL_NAME = f"univariate_DT_max-depth{MAX_DEPTH}_{train_reduc_factor_string}"
if CONSIDER_WS:
    MODEL_NAME += "_consider_ws"
MODEL_DESCRIPTION=MODEL_NAME+"_"+FACTORS_FOLDER
print(MODEL_DESCRIPTION)

univariate_DT_max-depthNone_training_factors=ti4-ct4-x/D4-y/D4_discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01


In [3]:
train_dataset, valid_dataset, test_dataset = get_wake_datasets(DATA_FOLDER,
                                                               consider_ws=CONSIDER_WS,
                                                               coords_as_input=COORDS_AS_INPUT,
                                                               #train_perc=0.6,
                                                               #validation_perc=0.2,
                                                               #test_perc=0.2
                                                               input_var_to_train_reduction_factor=INPUT_VAR_TO_TRAIN_REDUCTION_FACTOR,
                                                               #input_var_to_train_ranges=INPUT_VAR_TO_TRAIN_RANGES,
                                                               )

train_x, train_y = train_dataset.inputs, train_dataset.outputs
print("Train shapes: ", train_x.shape, train_y.shape)

#grid_size = train_dataset.num_cells
#print(f"{grid_size=}")

/Users/niccolomorabito/Library/CloudStorage/GoogleDrive-morabito.1808746@studenti.uniroma1.it/My Drive/BDMA/Semester4 Thesis/Ainslie-surrogate/utils/data_loaders.py:328: UserWarning: 
Ignoring percentages of train-valid-test split (train_perc=0.8, valid_perc=0, test_perc=0.2)
and using the reduction factors for the training set instead:
{'ti': 4, 'ct': 4, 'x/D': 4, 'y/D': 4}
  warnings.warn(


Incomplete field, creating DeficitDatasets...
Train shapes:  torch.Size([246400, 4]) torch.Size([246400, 1])


In [4]:
train_x, train_y = train_dataset.inputs, train_dataset.outputs
print("Train shapes: ", train_x.shape, train_y.shape)
train_size = train_x.shape[0]

test_x, test_y = test_dataset.inputs, test_dataset.outputs
print("Test shapes: ", test_x.shape, test_y.shape)
test_size = test_x.shape[0]

valid_x, valid_y = valid_dataset.inputs, valid_dataset.outputs
print("Valid shapes: ", valid_x.shape, valid_y.shape)
valid_size = valid_x.shape[0]

size = train_size + test_size + valid_size
print(train_size/size, test_size/size, valid_size/size)

Train shapes:  torch.Size([246400, 4]) torch.Size([246400, 1])
Test shapes:  torch.Size([30699200, 4]) torch.Size([30699200, 1])
Valid shapes:  torch.Size([30699200, 4]) torch.Size([30699200, 1])
0.003997093023255814 0.4980014534883721 0.4980014534883721


# Training

In [5]:
model = DecisionTreeRegressor(max_depth=MAX_DEPTH)
model.fit(train_x, train_y)
#tree_text = export_text(model)
#print(tree_text)

DecisionTreeRegressor()

In [6]:
importances = model.feature_importances_
for featurenum, importance in enumerate(importances):
    featurename = train_dataset.featurenum_to_featurename(featurenum)
    print(f"{featurename} feature importance={importance}")

ti feature importance=0.21742452223885578
ct feature importance=0.20595462053730568
x/D feature importance=0.14208687922001287
y/D feature importance=0.43453397800382565


# Evaluation and testing

In [7]:
save_results = True

# evaluation on trainset
evaluate_model(model, data=(train_x, train_y), data_type='train',
               model_description=MODEL_DESCRIPTION,
               save_results=save_results)

print("\n")

# evalution on testset
evaluate_model(model, data=(test_x, test_y), data_type='test',
               model_description=MODEL_DESCRIPTION,
               save_results=save_results)

Train results for univariate_DT_max-depthNone_training_factors=ti4-ct4-x/D4-y/D4_discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01
r2_score=0.9999999999999997
explained_variance_score=0.9999999999999997
mean_squared_error=4.876158504251731e-19
mean_absolute_error=2.971966705830031e-11
median_absolute_error=0.0
mean_absolute_percentage_error=0.032120654010807624
peak_signal_noise_ratio=183.11922185661726
Prediction time=1.3713519294540603e-07s


Test results for univariate_DT_max-depthNone_training_factors=ti4-ct4-x/D4-y/D4_discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01
r2_score=0.9445645048477482
explained_variance_score=0.944677141460631
mean_squared_error=6.172517001187271e-05
mean_absolute_error=0.0028885366207865863
median_absolute_error=0.0008568685734644532
mean_absolute_percentage_error=0.6463659247372647
peak_signal_noise_ratio=42.095377052013525
Prediction time=5.5062023416730804e-08s


In [8]:
num_cells = test_dataset.num_cells
num_fields = len(test_dataset)//num_cells
field_indices = list(range(num_fields))
random.shuffle(field_indices)

for field_idx in field_indices[:10]:
    ti, ct, ws, wake_field, predicted_wake_field = test_dataset.get_parameters_for_plotting_univariate(model, field_idx)
    plotting.plot_maps(test_dataset.X_grid, test_dataset.Y_grid, wake_field, predicted_wake_field, ti, ct, ws, error_to_plot='absolute')

AttributeError: 'DeficitDataset' object has no attribute 'num_cells'